# Imports

In [27]:
import numpy as np
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn
import math
# import cv2
import os
import time
from functools import wraps
from colorama import Fore
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotVisibleException
from requests import ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError

In [29]:
def retry(ExceptionToCheck, tries=4, delay=3, backoff=2, logger=None):
    """Retry calling the decorated function using an exponential backoff.

    http://www.saltycrane.com/blog/2009/11/trying-out-retry-decorator-python/
    original from: http://wiki.python.org/moin/PythonDecoratorLibrary#Retry

    :param ExceptionToCheck: the exception to check. may be a tuple of
        exceptions to check
    :type ExceptionToCheck: Exception or tuple
    :param tries: number of times to try (not retry) before giving up
    :type tries: int
    :param delay: initial delay between retries in seconds
    :type delay: int
    :param backoff: backoff multiplier e.g. value of 2 will double the delay
        each retry
    :type backoff: int
    :param logger: logger to use. If None, print
    :type logger: logging.Logger instance
    """
    def deco_retry(f):

        @wraps(f)
        def f_retry(*args, **kwargs):
            mtries, mdelay = tries, delay
            while mtries > 1:
                try:
                    return f(*args, **kwargs)
                except ExceptionToCheck:
                    msg = "%s, Retrying in %d seconds..." % (str(ExceptionToCheck), mdelay)
                    if logger:
                        #logger.exception(msg) # would print stack trace
                        logger.warning(msg)
                    else:
                        print(msg)
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
            return f(*args, **kwargs)

        return f_retry  # true decorator

    return deco_retry  

## Dataset

In [28]:
def createDailyDataset(text=False):
  import pandas as pd
  from tqdm import tqdm
  import ast
  import numpy as np

  tqdm.pandas()
  print("Reading data....")
  path = "../../data(fixed)/"

  df = pd.read_csv(f"{path}phelps_dataset(defense).csv")
  df['year'], df['month'], df['day'] = df['Year'], df['Month'], df['Day']
  date_df = pd.to_datetime(df[['year', 'month', 'day']])

  df = pd.read_csv(f"{path}phelps_dataset(defense).csv")
  df = df.drop(df.columns[0], axis=1)
  df['date'] = date_df

  df = df.sort_values('date', ascending=True).reset_index(drop=True)
  print(len(df.loc[(df['summary']=='Add a Plot\xa0»')]))
  df = df.loc[(df['summary']!='Add a Plot\xa0»')].reset_index(drop=True)

  df = df.sort_values('date', ascending=True).reset_index(drop=True)
  df = df.drop(columns=['max_theatres','plot', 'star_anchs'])

  df = df.loc[(df['Year']>=2000)&(df['Year']<=2019)].reset_index(drop=True)

  '''
    CREATE CLASSIFICATION COLUMN
  '''
  y = []
  '''
    Less than 1m
    1m - 20m
    20m - 60m
    60m - 200m
    200m +
  '''
  for value in df['box_office']:
    value = float(value)
    if value < 1000000:
      y.append(0)
    elif value >= 1000000 and value < 20000000:
      y.append(1)
    elif value >= 20000000 and value < 60000000:
      y.append(2)
    elif value >= 60000000:
      y.append(3)
  df['classification'] = y
  
  for idx, row in enumerate(df.itertuples()):
    classification = row.classification
    if np.isnan(row.original_budget):
      budget_average = df.loc[(df['classification']==classification)]['original_budget'].mean()
#       df.set_value(idx, 'original_budget', budget_average)
      df.at[idx, 'original_budget'] = budget_average
    if np.isnan(row.original_theatres):
      theatres_average = df.loc[(df['classification']==classification)]['original_theatres'].mean()
#       df.set_value(idx, 'original_theatres', theatres_average)
      df.at[idx, 'original_theatres']=theatres_average
    
  df['budget'], df['theatres'] = df['original_budget'], df['original_theatres']
  df = df.drop(columns=['original_budget', 'original_theatres'])



  if text==True:
    text_columns = ['summary', 'genre_cat']
    # re order to put categorical columns first
    categories = ['holiday','mpaa','main_language','sequel_binary','main_prod','distributor','brand','main_genre','pre_release_screening']
    ordered_columns =  text_columns + categories + [ elem for elem in df.columns if elem not in categories+text_columns] 
    df = df[ordered_columns]
  #   breakpoint()
    df['genre_cat'].fillna("['Unknown']", inplace = True)
    df['genre_cat'] = df['genre_cat'].apply(lambda x: ' '.join(ast.literal_eval(x)))
  else:
    df = df.drop(columns=['genre_cat', 'summary'])
    # re order to put categorical columns first
    categories = ['holiday','mpaa','main_language','sequel_binary','main_prod','distributor','brand','main_genre','pre_release_screening']
    ordered_columns =  categories + [ elem for elem in df.columns if elem not in categories] 
    df = df[ordered_columns]
    
  df = df.dropna().reset_index(drop=True)
  for val in df.columns:
    try:
      df[val] = df[val].astype(np.double)
    except:
      continue
  for val in categories:
    df[val] = df[val].astype('category')
  df['classification'] = df['classification'].apply(lambda x: int(x))
  print("\n")
  print("Complete!")


  daily_df = pd.read_csv(f"{path}daily_data.csv", index_col=0)
  daily_df = daily_df.drop(columns=['distributor'])

  
  df = pd.merge(df, daily_df, how='left', on=['name','profile'])

  df = df.dropna(subset=['opening_weekend_gross'])
  df = df.reset_index(drop=True)
  
  df = df.drop(columns=['main_language', 'main_prod', 'distributor', 'brand', 'main_genre'])
  df['sequel_binary'], df['pre_release_screening'] = df['sequel_binary'].astype(int), df['pre_release_screening'].astype(int)

  categories = ['holiday','mpaa']
  df = pd.get_dummies(df, columns=categories)

  return df

In [3]:
df = createDailyDataset()

/Users/brianphelps/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Reading data....
29


Complete!


In [4]:
df['url'] = 'nan'

In [32]:
df = pd.read_csv("./url_dataset.csv")

In [10]:
class GoogleSpider(object):
  def __init__(self, header):
    """
      Crawl Google search results

      This class is used to crawl Google's search results using requests and BeautifulSoup.
    """
    super().__init__()
    self.headers = {
      'User-Agent': header,
      'Host': 'www.google.com',
      'Referer': 'https://www.google.com/'
    }

  def __get_source(self, url: str) -> requests.Response:
    """
      Get the web page's source code

      Args:
        url (str): The URL to crawl

      Returns:
        requests.Response: The response from URL
    """
    return requests.get(url, headers=self.headers)

  def search(self, query: str) -> list:
      """
        Search Google

        Args:
          query (str): The query to search for

        Returns:
          list: The search results
      """
      # Get response
      response = self.__get_source(f'https://www.google.com/{query}')
      # Initialize BeautifulSoup
      soup = BeautifulSoup(response.text, 'html.parser')
      # Get the result containers
      result_containers = soup.findAll('a')
      # Final results list
      results = []
      for anchor in result_containers:
        try:
          href = anchor['href']
          if 'rottentomatoes.com/m/' in href:
            url = href
            return url
        except:
          pass
      return np.nan

In [33]:
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
import urllib.parse
import numpy as np
# Import dependencies
from urllib.parse import quote
from pprint import pprint
from googlesearch import search 
import random

In [34]:
df = df.replace(np.nan, 'nan')

In [42]:
chromedriver_path = "/Users/brianphelps/Desktop/chromedriver"

# OPTIONS

# options.add_argument("headless")

In [45]:
data = []

In [129]:
import random

In [190]:
indexes = []
for idx, row in enumerate(df.itertuples()):
  if row.url=='nan':
    indexes.append(idx)

In [193]:
df = df.reset_index(drop=True)

In [192]:
df = df.drop(indexes[:3])

In [208]:
result_dataframe = pd.DataFrame()

API_KEY = '8ed1c1a9d314e36eb0baf42b055e46e6'
counter=0
user_agent = UserAgent()
for row in df.itertuples():
  if row.url=='nan':
    time.sleep(2)
    counter+=1
    name = urllib.parse.quote(row.name)
    year = int(row.Year)
    profile = row.profile
    
    headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:79.0) Gecko/20100101 Firefox/79.0',
            'Host': 'www.google.com',
            'Referer': 'https://www.google.com/'
        }
    
    query = f"/search?q={name}%20{year}%20site:rottentomatoes.com"
    user_data_path = f"/Users/brianphelps/Library/Application Support/Google/Chrome/Defaultzorro{counter+9000}"
    
    options = webdriver.ChromeOptions()
    options.add_argument("no-sandbox")
    options.add_argument("disable-dev-shm-usage")
    options.add_argument(f"user-data-dir={user_data_path}")
#     if counter > 3:
#       options.add_argument("headless")
    
    
    driver = webdriver.Chrome(f"{chromedriver_path}", chrome_options=options)

    @retry((ConnectionResetError,TimeoutException,ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError), tries=20, delay=2, backoff=2)
    def driver_reviews():
      driver.get(f"https://www.google.com{query}")
    driver_reviews()

    src = driver.page_source
    driver.quit()
    soup = BeautifulSoup(src, 'lxml')
    # Get the result containers
    result_containers = soup.findAll('a')
    # Final results list
    results = []
    search_anchors=True
    for anchor in result_containers:
      if search_anchors:
        try:
          href = anchor['href']
          if 'rottentomatoes.com/m/' in href:
            url = href
            print(f"{year}: {row.name}")
            print(url)
            print(f"{counter}/{len(df)}")
            clear_output(wait=True)
            temp = pd.DataFrame({
              'profile': [profile],
              'url': [url]
            })
            result_dataframe = result_dataframe.append(temp, ignore_index=True)
            search_anchors=False
            continue
        except:
          url = np.nan
          print(f"{year}: {row.name}")
          print(url)
          print(f"{counter}/{len(df)}")
          clear_output(wait=True)
          temp = pd.DataFrame({
            'profile': [profile],
            'url': [url]
          })
          result_dataframe = result_dataframe.append(temp, ignore_index=True)


2019: Before You Know It
https://www.rottentomatoes.com/m/before_you_know_it_2019
43/3545


In [209]:
result_dataframe = result_dataframe.dropna()
result_dataframe = result_dataframe.reset_index(drop=True)
print(result_dataframe.shape)

(43, 2)


In [210]:
df = df.reset_index(drop=True)

In [211]:
for row in result_dataframe.itertuples():
  profile = row.profile
  index = df.loc[df['profile']==profile].index[0]
  try:
    df.at[index, 'url']=row.url
  except:
    breakpoint()

In [222]:
len(df.loc[(df['url']!='nan')])

3545

In [213]:
df.to_csv("url_dataset.csv")

In [215]:
df['url']

0       https://www.rottentomatoes.com/m/snow_falling_...
1               https://www.rottentomatoes.com/m/magnolia
2            https://www.rottentomatoes.com/m/next_friday
3       https://www.rottentomatoes.com/m/girl_interrupted
4       https://www.rottentomatoes.com/m/1093641-hurri...
                              ...                        
3540               https://www.rottentomatoes.com/m/saaho
3541          https://www.rottentomatoes.com/m/todos_caen
3542    https://www.rottentomatoes.com/m/dont_let_go_2019
3543        https://www.rottentomatoes.com/m/bennetts_war
3544    https://www.rottentomatoes.com/m/before_you_kn...
Name: url, Length: 3545, dtype: object

In [219]:
links = df[['name', 'profile', 'url']]

In [221]:
links.to_csv("./rotten_tomatoe_links.csv")